In [1]:
!nvidia-smi

Fri May 24 15:05:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [7]:
from peft import PeftModel, PeftConfig

In [8]:
peft_model_id = "Dohun99/mechuri-koalpaca-polyglot-5.8b-2140step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 4096)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-27): 28 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=12288, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

In [9]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [10]:
gen('점심 추천해줘')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:581: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 점심 추천해줘

### 답변: 매콤한 짜장면과 탕수육도 함께 먹어보세요. 점심 외식으로 가볍게 먹기에 좋아요. 맛있는 점심을 즐기세요!

### 답변: 점심으로는 매콤한 짜장면과 탕수육을 추천드려


In [11]:
!pip install gradio

In [18]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

chatbot_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

def food_recommendation(messages, user_message):
    # if "점심" in user_message:
    #     bot_message = "점심으로 파스타는 어떠세요?"
    # elif "저녁" in user_message:
    #     bot_message = "저녁으로 스테이크는 어떠세요?"
    # elif "디저트" in user_message:
    #     bot_message = "디저트로 아이스크림은 어떠세요?"
    # else:
    #     bot_message = "어떤 음식을 추천해 드릴까요?"

    bot_response = chatbot_pipeline(user_message, max_length=200, num_return_sequences=1)[0]['generated_text']
    bot_message = bot_response[len(user_message):].strip()
    bot_message = bot_message.replace("답변:", "").strip()


    messages.append((user_message,bot_message))
    return messages, ""

# Gradio 인터페이스 정의
with gr.Blocks(css="""
    @font-face {
    font-family: 'Katuri';
    src: url('https://cdn.jsdelivr.net/gh/projectnoonnu/noonfonts_13@1.0/Katuri.woff') format('woff');
    font-weight: normal;
    font-style: normal;
    }
    @import url('//fonts.googleapis.com/earlyaccess/notosanskr.css');


    .user {
      background-color: #F7CAC9;
      padding: 10px;
      border-radius: 20px 20px 0 20px;
      align-self: flex-end;
      margin: 5px 0;
      font-family: 'Noto Sans KR';
    }
    .bot {
      background-color: #ABCEF0;
      padding: 10px;
      border-radius: 20px 20px 20px 0;
      align-self: flex-start;
      margin: 5px 0;
      font-family: 'Noto Sans KR';
      font-weight: normal !important;
      font-size: 14px !important;
    }

    .gr-chat-message {
        background-color: white;
        padding: 10px;
        border-radius: 15px;
        box-shadow: 1px 1px 5px rgba(0, 0, 0, 0.1);
        margin: 5px;
        position: relative;
        font-weight: normal !important;
        max-width: 100%; /* 메시지의 최대 너비를 100%로 설정 */
        word-break: break-word; /* 단어가 너무 길면 줄 바꿈 */
        white-space: pre-wrap; /* 공백을 유지하면서 줄 바꿈 */
    }
    .gr-chat-message.bot {
        background-color: #F7CAC9;
        align-self: flex-start;
        font-weight: normal !important;
    }
    .gr-chat-message.user {
        background-color: #FFD740;
        align-self: flex-end;
        font-weight: normal !important;
    }
    .user .timestamp, .bot .timestamp {
        font-size: 12px;
        color: #666;
        margin-top: 5px;
    }
    .send-button {
        background-color: black !important;
        color: black !important;
        width: 10px;
        height: 50px;
        font-size: 14px !important;
        border: none !important;
        border-radius: 5px !important;
        cursor: pointer !important;
    }

    .send-button:hover {
        background-color: #45a049 !important;
    }
    .chat-container {
        background: linear-gradient(to right, #6a11cb 0%, #2575fc 100%);
    }
""") as demo:
    gr.Markdown("<h1 style=\"text-align: center; color: black; font-family: 'Katuri', sans-serif; font-size: 32px\">🍽️ 메 추  리 🍽️</h1>")

    chatbot = gr.Chatbot()

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="음식 종류나 끼니를 입력하세요. 예: 점심, 저녁, 디저트")
        txt.submit(food_recommendation, [chatbot, txt], [chatbot, txt])
        submit_btn = gr.Button("보내기", elem_classes="send-button")

    def on_submit(user_message, state):
        return food_recommendation(state, user_message)

    submit_btn.click(on_submit, [txt, chatbot], [chatbot, txt])

    with gr.Row():
        gr.Markdown("<p style='text-align: center;'>챗봇에게 원하는 음식 종류나 끼니를 입력하면 추천해 드립니다!</p>")

demo.launch()

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6bab1bd40a51b09f2d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
